In [1]:
# %% [markdown]
# # Lab 5: LangGraph Basics - Your First Graph
#
# **Goal:** Build a simple, linear workflow for processing a resume using the fundamental components of LangGraph.
#
# ---

# %% [markdown]
# ## Part A: Setup and Installation
#
# First, we need to install the necessary libraries. `langgraph` is the core library, and `langchain-openai` allows us to use OpenAI's models.

# %%
# Install required packages
!pip install langgraph langchain-openai --quiet
print("Libraries installed.")

# Basic imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# %% [markdown]
# ---
# ## Part B: Define the State
#
# The "state" is a central concept in LangGraph. It's a Python object that holds all the data that will be passed between the steps (nodes) in our graph.
#
# For our resume processor, the state will hold the resume text, the skills we extract, the score we assign, and the final decision. We define its structure using `TypedDict`.

# %%
# Define what data flows through our graph
class ResumeState(TypedDict):
    """
    Represents the state of our recruitment workflow.
    """
    resume_text: str
    skills: List[str]
    score: int
    decision: str

print("ResumeState defined.")

# %% [markdown]
# **Key Concept:** The state is the memory of your workflow. Every node can read from it and write to it.
#
# ---

# %% [markdown]
# ## Part C: Create the Nodes
#
# Nodes are the building blocks of the graph. Each node is a Python function that performs a specific task. A node function always takes the `state` as its input and returns a dictionary with the values to update in the state.
#
# We will create three nodes:
# 1.  `parse_resume`: To extract skills from the resume text.
# 2.  `score_candidate`: To assign a score based on the extracted skills.
# 3.  `make_decision`: To make a final decision based on the score.

# %%
def parse_resume(state: ResumeState) -> dict:
    """
    A node that extracts a predefined list of skills from the resume text.
    """
    print("---NODE: PARSING RESUME---")
    resume = state["resume_text"]

    # Simple skill extraction (we'll improve this later)
    found_skills = []
    if "python" in resume.lower():
        found_skills.append("Python")
    if "java" in resume.lower():
        found_skills.append("Java")
    if "management" in resume.lower():
        found_skills.append("Management")

    print(f"Parsed skills: {found_skills}")

    # Return a dictionary with the state fields to update
    return {"skills": found_skills}

def score_candidate(state: ResumeState) -> dict:
    """
    A node that gives the candidate a score based on the number of skills found.
    """
    print("---NODE: SCORING CANDIDATE---")
    skills = state["skills"]

    # Simple scoring logic: 20 points per skill
    score = len(skills) * 20
    print(f"Candidate score: {score}")

    return {"score": score}

def make_decision(state: ResumeState) -> dict:
    """
    A node that makes a final hiring decision based on the score.
    """
    print("---NODE: MAKING DECISION---")
    score = state["score"]

    if score >= 60:
        decision = "Interview"
    elif score >= 40:
        decision = "Second Review"
    else:
        decision = "Reject"

    print(f"Final Decision: {decision}")
    return {"decision": decision}

# %% [markdown]
# ---
# ## Part D: Build and Test the Graph
#
# Now we assemble the pieces into a complete workflow.
#
# 1.  **Instantiate `StateGraph`**: We create a graph object, telling it the structure of our state.
# 2.  **Add Nodes**: We add the functions we created as nodes, giving each a unique name.
# 3.  **Add Edges**: We connect the nodes to define the flow of control. We also define the `START` and `END` points of the graph.
# 4.  **Compile**: We compile the graph into a runnable application.

# %%
# 1. Create the StateGraph instance
workflow = StateGraph(ResumeState)

# 2. Add the nodes to the graph
workflow.add_node("parse", parse_resume)
workflow.add_node("score", score_candidate)
workflow.add_node("decide", make_decision)

# 3. Add the edges to define the flow
workflow.set_entry_point("parse") # The graph starts at the 'parse' node
workflow.add_edge("parse", "score")
workflow.add_edge("score", "decide")
workflow.add_edge("decide", END) # The 'decide' node is the final step

# 4. Compile the graph into a runnable app
app = workflow.compile()
print("\nGraph compiled successfully!")

# %% [markdown]
# ### Testing the Graph
#
# Let's run our compiled graph with some sample input. We provide the initial state, and the graph will execute the nodes in the sequence we defined.

# %%
# Define the initial input for the graph
initial_state = {
    "resume_text": "Experienced Python developer with strong Java and project management experience.",
    "skills": [],
    "score": 0,
    "decision": ""
}

# Invoke the graph and print the final result
final_result = app.invoke(initial_state)

print("\n---FINAL RESULT---")
print(final_result)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 15.4 MB/s eta 0:00:00
Libraries installed.
ResumeState defined.

Graph compiled successfully!
---NODE: PARSING RESUME---
Parsed skills: ['Python', 'Java', 'Management']
---NODE: SCORING CANDIDATE---
Candidate score: 60
---NODE: MAKING DECISION---
Final Decision: Interview

---FINAL RESULT---
{'resume_text': 'Experienced Python developer with strong Java and project management experience.', 'skills': ['Python', 'Java', 'Management'], 'score': 60, 'decision': 'Interview'}
